In [1]:
from itertools import product
from typing import Tuple, Literal, Callable

import numpy as np
import scipy as sp
import numpy.linalg as linalg
import scipy.linalg
import pandas as pd
import plotly.graph_objects as go

from enum import Enum

# Util

In [2]:
matrix_size_name = "matrix_size"
matrix_a_condition_name = "cond(A)"
matrix_l_condition_name = "cond(L)"
matrix_u_condition_name = "cond(U)"
error_column_name = "error"
alpha_column_name = "alpha"
regulirized_matrix_a_condition_name = "cond(A + alpha * E)"
regulirized_matrix_l_condition_name = "cond(reg(L))"
regulirized_matrix_u_condition_name = "cond(reg(U))"
regulirized_error_column_name = "reg_error"

class ConditionNumberType(Enum):
    SpectralCondition = "cond_s"
    VolumCondition = "cond_v"
    AngleCondition = "cond_a"

In [3]:
def calculate_spectral_condition_number(matrix):
    return linalg.cond(matrix)


def calculate_volume_condition_number(matrix):
    volume_denominator = abs(linalg.det(matrix))

    volume_numerator = 1
    for row in matrix:
        volume_numerator *= linalg.norm(row)

    return volume_numerator / volume_denominator


def calculate_angle_condition_number(matrix):
    candidates = []
    for row, column in zip(matrix, linalg.inv(matrix).T):
        candidates.append(linalg.norm(row) * linalg.norm(column))

    return max(candidates)


def calculate_condition_number(condition_number_type: ConditionNumberType, matrix):
    return {
        ConditionNumberType.AngleCondition: calculate_angle_condition_number,
        ConditionNumberType.SpectralCondition: calculate_spectral_condition_number,
        ConditionNumberType.VolumCondition: calculate_volume_condition_number,
    }[condition_number_type](matrix)

In [4]:
def generate_matrix(element_factory, size):
    return np.array(
        [
            [element_factory(row, column) for column in range(1, size + 1)]
            for row in range(1, size + 1)
        ]
    )

In [5]:
def draw_compare_plot_for_size_and_cond(data):
    fig = go.Figure()

    for matrix_condition_name in [matrix_a_condition_name, matrix_l_condition_name, matrix_u_condition_name]:
        fig.add_scatter(x=data[matrix_size_name], y=data[matrix_condition_name], name=matrix_condition_name)

    fig.update_xaxes(title='Размер матрицы')
    fig.update_yaxes(title='Число обусловленности', tickformat='.2e')
    fig.update_layout(title='Зависимость числа обусловленности от размера матрицы')

    fig.show()

In [6]:
def draw_compare_plot_for_size_and_error(data):
    fig = go.Figure()

    fig.add_scatter(x=data[matrix_size_name], y=data[error_column_name])

    fig.update_xaxes(title='Размер матрицы')
    fig.update_yaxes(title='Ошибка', tickformat='.2e')
    fig.update_layout(title='Зависимость ошибки от размера матрицы')

    fig.show()

In [7]:
def get_lup_decomposition(matrix):
    return sp.linalg.lu(matrix)

In [8]:
def forward_substitution(l, right_part):
    size = l.shape[0]

    solution = []
    for i in range(size):
        solution.append((right_part[i] - np.dot(l[i, :i], solution).sum()) / l[i][i])

    return np.array(solution)


def back_substitution(u, right_part):
    return forward_substitution(np.flip(u, axis=[0, 1]), right_part[::-1])[::-1]


def solve_using_lu_decompostion(matrix, right_part):
    p, l, u = get_lup_decomposition(matrix)
    y = forward_substitution(l, (np.linalg.inv(p) @ right_part))
    return back_substitution(u, y)

In [10]:
def run_benchmark(
        matrix, 
        right_part, 
        expected_solution, 
        condition_number_type: ConditionNumberType = ConditionNumberType.AngleCondition
):
    _, l, u = get_lup_decomposition(matrix)

    matrix_condition_number = calculate_condition_number(condition_number_type, matrix)
    l_condition_number = calculate_condition_number(condition_number_type, l)
    u_condition_number = calculate_condition_number(condition_number_type, u)

    actual_solution = solve_using_lu_decompostion(matrix, right_part)

    solution_without_decomposition = sp.linalg.solve(matrix, right_part)

    return pd.Series(
        [
            matrix_condition_number,
            l_condition_number,
            u_condition_number,
            linalg.norm(expected_solution - actual_solution),
            linalg.norm(expected_solution - solution_without_decomposition)
        ]
    )

# Хорошие примеры из Пакулиной

In [11]:
def get_pakulina_1_linear_system():
    matrix = np.array([
        [3.278164, 1.046583, -1.378574],
        [1.046583, 2.975937, 0.934251],
        [-1.378574, 0.934251, 4.836173],
    ])
    solution = np.ones(3)
    right_part = matrix.dot(solution)
    return matrix, right_part, solution

In [12]:
def get_pakulina_6_linear_system():
    matrix = np.array([
        [9.016024, 1.082197, 2.783575],
        [1.08219, 6.846595, 0.647647],
        [-2.78357, 0.647647, 5.432541],
    ])
    solution = np.ones(3)
    right_part = matrix.dot(solution)
    return matrix, right_part, solution

In [13]:
def get_pakulina_11_linear_system():
    matrix = np.array([
        [6.687233, 0.80267, 2.06459],
        [0.8026, 5.07816, 0.48037],
        [-2.06459, 0.48037, 4.02934],
    ])
    solution = np.ones(3)
    right_part = matrix.dot(solution)
    return matrix, right_part, solution

In [14]:
good_data = pd.DataFrame.from_dict({
    1: run_benchmark(*get_pakulina_1_linear_system()),
    6: run_benchmark(*get_pakulina_6_linear_system()),
    11: run_benchmark(*get_pakulina_11_linear_system()),
}, orient='index')
good_data.columns = [matrix_a_condition_name, matrix_l_condition_name, matrix_u_condition_name, error_column_name, "error_without_decomp"]
good_data

,cond(A),cond(L),cond(U),error,error_without_decomp
1,1.906892,1.203120,1.270739,7.447602e-16,7.447602e-16
6,1.065856,1.058710,1.053435,2.482534e-16,3.330669e-16
11,1.065853,1.058708,1.053435,3.140185e-16,2.220446e-16


# Плохие примеры (матрица Гильберта)

In [17]:
def get_hilbert_linear_system(size, expected_solution=None):
    matrix = np.around(generate_matrix(lambda row, column: 1 / (row + column - 1), size), 10)
    
    expected_solution = expected_solution if expected_solution is not None else np.ones(size)
    right_part = np.dot(matrix, expected_solution)

    return matrix, right_part, expected_solution

In [18]:
hilbert_data = pd.DataFrame(range(2, 100), columns=[matrix_size_name])
hilbert_data[[matrix_a_condition_name, matrix_l_condition_name, matrix_u_condition_name, error_column_name, "error_without_decomp"]] = hilbert_data.apply(
    lambda row: run_benchmark(*get_hilbert_linear_system(row[matrix_size_name])),
    axis=1,
)
hilbert_data.sort_values(by=error_column_name, ascending=False)

,matrix_size,cond(A),cond(L),cond(U),error,error_without_decomp
69,71,2.727659e+13,11.851698,9.396397e+05,3.801120e-02,3.134550e-02
81,83,3.574778e+11,13.218905,1.231788e+06,8.619459e-04,2.005467e-04
25,27,1.731782e+11,6.228150,3.036746e+04,5.643279e-04,3.500657e-04
26,28,1.860718e+11,6.516365,3.425235e+04,4.938520e-04,4.357577e-04
80,82,1.297828e+11,13.062564,1.203102e+06,4.422038e-04,5.896167e-04
...,...,...,...,...,...,...
4,6,2.441337e+06,2.388348,7.774785e+00,4.564831e-10,8.558862e-10
3,5,9.515739e+04,2.438588,4.304053e+00,1.278924e-11,1.507071e-13
2,4,4.020914e+03,2.173067,2.660409e+00,1.110223e-15,9.452556e-15
1,3,1.728872e+02,1.500000,1.634693e+00,1.093443e-15,1.389554e-14


In [19]:
draw_compare_plot_for_size_and_cond(hilbert_data)

In [20]:
draw_compare_plot_for_size_and_error(hilbert_data)

# Регуляризация

In [21]:
def run_benchmark_with_regularization(
        matrix,
        right_part,
        expected_solution,
        alpha,
        condition_number_type: ConditionNumberType = ConditionNumberType.AngleCondition
):
    regularized_matrix = matrix + alpha * np.eye(matrix.shape[0])

    return pd.concat(
        [
            run_benchmark(matrix, right_part, expected_solution, condition_number_type),
            run_benchmark(regularized_matrix, right_part, expected_solution, condition_number_type),
        ],
        ignore_index=True,
    )

In [23]:
regularization_data = pd.DataFrame(product(range(15, 100), [10 ** -i for i in range(1, 13)]), columns=[matrix_size_name, alpha_column_name])
regularization_data[
    [
        matrix_a_condition_name, matrix_l_condition_name, matrix_u_condition_name, error_column_name, "error",
        regulirized_matrix_a_condition_name, regulirized_matrix_l_condition_name, regulirized_matrix_u_condition_name, regulirized_error_column_name, "error"
    ]
] = regularization_data.apply(
    lambda row: run_benchmark_with_regularization(*get_hilbert_linear_system(int(row[matrix_size_name])), alpha=row[alpha_column_name]),
    axis=1,
)

regularization_data.sort_values(by=[regulirized_error_column_name])

,matrix_size,alpha,cond(A),cond(L),cond(U),error,cond(A + alpha * E),cond(reg(L)),cond(reg(U)),reg_error
23,16,1.000000e-12,2.905487e+09,3.987557,1.451252e+03,0.000003,2.848736e+09,3.983247,1451.064439,0.000002
10,15,1.000000e-11,2.763266e+09,4.314435,1.333303e+03,0.000002,2.973588e+09,4.243693,1314.588613,0.000002
11,15,1.000000e-12,2.763266e+09,4.314435,1.333303e+03,0.000003,2.745567e+09,4.306196,1331.396012,0.000003
22,16,1.000000e-11,2.905487e+09,3.987557,1.451252e+03,0.000004,2.650415e+09,3.955770,1449.379601,0.000004
82,21,1.000000e-11,1.437210e+10,4.991602,1.175430e+04,0.000006,6.531661e+09,4.827413,11590.478216,0.000005
...,...,...,...,...,...,...,...,...,...,...
960,95,1.000000e-01,4.243491e+09,17.609919,1.577324e+06,1.789997,6.734194e+00,1.272030,1.700797,1.789997
972,96,1.000000e-01,1.688514e+10,15.771407,1.606054e+06,1.799398,6.734688e+00,1.272030,1.701056,1.799398
984,97,1.000000e-01,9.308828e+10,16.623039,1.806428e+06,1.808751,6.735172e+00,1.272030,1.701311,1.808751
996,98,1.000000e-01,7.069957e+09,18.210555,1.842588e+06,1.818055,6.735647e+00,1.272030,1.701560,1.818055


In [26]:
def find_best_alpha(size, data):
    data = data[data[matrix_size_name] == size]
    data.reset_index(drop=True, inplace=True)

    min_error_arg = data[regulirized_error_column_name].argmin()

    min_error = data.loc[min_error_arg, regulirized_error_column_name]
    alpha = data.loc[min_error_arg, alpha_column_name]

    return pd.Series(
        [
            data.loc[min_error_arg, matrix_a_condition_name], 
            data.loc[min_error_arg, regulirized_matrix_a_condition_name], 
            alpha, 
            min_error,
            data.loc[min_error_arg, error_column_name]
        ]
    )

In [29]:
regularization_best_data = pd.DataFrame(range(15, 100), columns=[matrix_size_name])
regularization_best_data[
    [
        matrix_a_condition_name, 
        regulirized_matrix_a_condition_name, 
        alpha_column_name, 
        regulirized_error_column_name,
        error_column_name
    ]
] = regularization_best_data.apply(
    func=lambda row: find_best_alpha(row[matrix_size_name], regularization_data),
    axis=1,
)

regularization_best_data

,matrix_size,cond(A),cond(A + alpha * E),alpha,reg_error,error
0,15,2.763266e+09,2.973588e+09,1.000000e-11,0.000002,0.000002
1,16,2.905487e+09,2.848736e+09,1.000000e-12,0.000002,0.000003
2,17,4.709685e+09,4.868018e+09,1.000000e-12,0.000006,0.000003
3,18,4.322703e+09,4.438588e+09,1.000000e-12,0.000006,0.000003
4,19,8.678518e+10,1.246350e+10,1.000000e-11,0.000027,0.000020
...,...,...,...,...,...,...
80,95,4.243491e+09,4.161102e+09,1.000000e-12,0.000014,0.000017
81,96,1.688514e+10,2.109636e+10,1.000000e-12,0.000017,0.000013
82,97,9.308828e+10,8.472507e+09,1.000000e-11,0.000044,0.000018
83,98,7.069957e+09,6.575438e+09,1.000000e-12,0.000022,0.000013


In [30]:
regularization_data = pd.DataFrame(product(range(15, 100), [10 ** -i for i in range(1, 13)]), columns=[matrix_size_name, alpha_column_name])
regularization_data[
    [
        matrix_a_condition_name, matrix_l_condition_name, matrix_u_condition_name, error_column_name, "error_without_decompose",
        regulirized_matrix_a_condition_name, regulirized_matrix_l_condition_name, regulirized_matrix_u_condition_name, regulirized_error_column_name, "error_without_decompose"
    ]
] = regularization_data.apply(
    lambda row: run_benchmark_with_regularization(*get_hilbert_linear_system(int(row[matrix_size_name]), np.random.rand(int(row[matrix_size_name]))), alpha=row[alpha_column_name]),
    axis=1,
)

regularization_data.sort_values(by=[regulirized_error_column_name])

,matrix_size,alpha,cond(A),cond(L),cond(U),error,error_without_decompose,cond(A + alpha * E),cond(reg(L)),cond(reg(U)),reg_error
35,17,1.000000e-12,4.709685e+09,4.098536,2.416383e+03,0.000001,0.005316,4.868018e+09,4.105877,2.414432e+03,0.005317
11,15,1.000000e-12,2.763266e+09,4.314435,1.333303e+03,0.000002,0.009809,2.745567e+09,4.306196,1.331396e+03,0.009809
107,23,1.000000e-12,1.124432e+10,6.926743,1.337392e+04,0.000004,0.012473,1.249549e+10,6.889206,1.336448e+04,0.012469
23,16,1.000000e-12,2.905487e+09,3.987557,1.451252e+03,0.000001,0.013524,2.848736e+09,3.983247,1.451064e+03,0.013523
239,34,1.000000e-12,1.320643e+10,6.975466,8.552111e+04,0.000027,0.018261,1.552887e+10,6.976425,8.540618e+04,0.018257
...,...,...,...,...,...,...,...,...,...,...,...
765,78,1.000000e-10,8.509500e+09,11.213462,1.099382e+06,0.000015,109.679836,1.544470e+11,13.914609,1.111820e+06,109.679839
81,21,1.000000e-10,1.437210e+10,4.991602,1.175430e+04,0.000006,227.829477,1.383918e+12,5.833838,1.237145e+04,227.828518
93,22,1.000000e-10,1.140840e+10,5.065154,1.042490e+04,0.000013,439.866596,1.013815e+12,5.931719,1.275206e+04,439.866255
429,50,1.000000e-10,1.067737e+10,10.326967,2.392717e+05,0.000017,1116.630987,7.752554e+11,13.756792,2.340212e+05,1116.630935


In [32]:
regularization_best_data = pd.DataFrame(range(15, 30), columns=[matrix_size_name])
regularization_best_data[
    [
        matrix_a_condition_name, 
        regulirized_matrix_a_condition_name, 
        alpha_column_name, 
        regulirized_error_column_name,
        error_column_name,
    ]
] = regularization_best_data.apply(
    func=lambda row: find_best_alpha(row[matrix_size_name], regularization_data),
    axis=1,
)

regularization_best_data

,matrix_size,cond(A),cond(A + alpha * E),alpha,reg_error,error
0,15,2.763266e+09,2.745567e+09,1.000000e-12,0.009809,0.000002
1,16,2.905487e+09,2.848736e+09,1.000000e-12,0.013523,0.000001
2,17,4.709685e+09,4.868018e+09,1.000000e-12,0.005317,0.000001
3,18,4.322703e+09,4.438588e+09,1.000000e-12,0.023620,0.000002
4,19,8.678518e+10,1.246350e+10,1.000000e-11,0.271788,0.000038
5,20,1.122898e+10,1.037008e+10,1.000000e-12,0.025147,0.000004
6,21,1.437210e+10,1.276320e+10,1.000000e-12,0.019135,0.000009
7,22,1.140840e+10,1.026542e+10,1.000000e-12,0.044431,0.000006
8,23,1.124432e+10,1.249549e+10,1.000000e-12,0.012469,0.000004
9,24,6.347834e+09,6.706288e+09,1.000000e-12,0.029062,0.000002
